In [1]:
import numpy as np
from Pyfhel import Pyfhel
import time

# Interger

In [2]:
print("Context and key setup")
HE = Pyfhel()
HE.contextGen(scheme='bfv', n=2**14, t_bits=20)
HE.keyGen()
print(HE)

Context and key setup
<bfv Pyfhel obj at 0x13f4dd173a0, [pk:Y, sk:Y, rtk:-, rlk:-, contx(n=16384, t=786433, sec=128, qi=[], scale=1.0, )]>


In [3]:
integer1 = np.array([127], dtype=np.int64)
integer2 = np.array([-2], dtype=np.int64)
ctxt1 = HE.encryptInt(integer1) # Encryption makes use of the public key
ctxt2 = HE.encryptInt(integer2) # For integers, encryptInt function is used.
print("Integer Encryption, ")
print("    int ",integer1,'-> ctxt1 ', type(ctxt1))
print("    int ",integer2,'-> ctxt2 ', type(ctxt2))

Integer Encryption, 
    int  [127] -> ctxt1  <class 'Pyfhel.PyCtxt.PyCtxt'>
    int  [-2] -> ctxt2  <class 'Pyfhel.PyCtxt.PyCtxt'>


In [4]:
print(ctxt1)
print(ctxt2)

<Pyfhel Ciphertext at 0x13f651599f0, scheme=bfv, size=2/2, noiseBudget=361>
<Pyfhel Ciphertext at 0x13f4dd7f2c0, scheme=bfv, size=2/2, noiseBudget=361>


In [6]:
ctxtSum = ctxt1 + ctxt2         # `ctxt1 += ctxt2` for inplace operation
ctxtSub = ctxt1 - ctxt2         # `ctxt1 -= ctxt2` for inplace operation
ctxtMul = ctxt1 * ctxt2         # `ctxt1 *= ctxt2` for inplace operation
print("Operating with encrypted integers")
print(f"Sum: {ctxtSum}")
print(f"Sub: {ctxtSub}")
print(f"Mult:{ctxtMul}")

Operating with encrypted integers
Sum: <Pyfhel Ciphertext at 0x13f6516c7c0, scheme=bfv, size=2/2, noiseBudget=360>
Sub: <Pyfhel Ciphertext at 0x13f4dccb040, scheme=bfv, size=2/2, noiseBudget=360>
Mult:<Pyfhel Ciphertext at 0x13f651aac70, scheme=bfv, size=3/3, noiseBudget=328>


In [7]:
resSum = HE.decryptInt(ctxtSum) # Decryption must use the corresponding function decryptInt.
resSub = HE.decryptInt(ctxtSub)
resMul = HE.decryptInt(ctxtMul)
print("Decrypting result:")
print("     addition:       decrypt(ctxt1 + ctxt2) =  ", resSum)
print("     substraction:   decrypt(ctxt1 - ctxt2) =  ", resSub)
print("     multiplication: decrypt(ctxt1 + ctxt2) =  ", resMul)

Decrypting result:
     addition:       decrypt(ctxt1 + ctxt2) =   [125   0   0 ...   0   0   0]
     substraction:   decrypt(ctxt1 - ctxt2) =   [129   0   0 ...   0   0   0]
     multiplication: decrypt(ctxt1 + ctxt2) =   [-254    0    0 ...    0    0    0]


# Float numbers

In [10]:
n_mults = 8

HE = Pyfhel(key_gen=True, context_params={
    'scheme': 'CKKS',
    'n': 2**14,         # For CKKS, n/2 values can be encoded in a single ciphertext.
    'scale': 2**30,     # Each multiplication grows the final scale
    'qi_sizes': [60]+ [30]*n_mults +[60] # Number of bits of each prime in the chain.
                        # Intermediate prime sizes should be close to log2(scale).
                        # One per multiplication! More/higher qi_sizes means bigger
                        #  ciphertexts and slower ops.
})
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()
HE.relinKeyGen()
print("CKKS context generation")
print(f"\t{HE}")

CKKS context generation
	<ckks Pyfhel obj at 0x215ff464df0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 30, 30, 30, 30, 30, 30, 30, 60], scale=1073741824.0, )]>


In [23]:
arr_x = np.arange(HE.n//2, dtype=np.float64)*0.1
arr_y = np.arange(HE.n//2, dtype=np.float64)*0.05

ptxt_x = HE.encodeFrac(arr_x)   # Creates a PyPtxt plaintext with the encoded arr_x
ptxt_y = HE.encodeFrac(arr_y)   # plaintexts created from arrays shorter than 'n' are filled with zeros.

#ctxt_x = HE.encryptPtxt(ptxt_x) # Encrypts the plaintext ptxt_x and returns a PyCtxt
#ctxt_y = HE.encryptPtxt(ptxt_y) #  Alternatively you can use HE.encryptFrac(arr_y)

ctxt_x = HE.encryptFrac(arr_x)
ctxt_y = HE.encryptFrac(arr_y)

print("Fixed-point Encoding & Encryption, ")
print("->\tarr_x ", arr_x,'\n\t==> ctxt_x ', ctxt_x)
print("->\tarr_y ", arr_y,'\n\t==> ctxt_y ', ctxt_y)

Fixed-point Encoding & Encryption, 
->	arr_x  [0.000e+00 1.000e-01 2.000e-01 ... 8.189e+02 8.190e+02 8.191e+02] 
	==> ctxt_x  <Pyfhel Ciphertext at 0x215a797e950, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	arr_y  [0.0000e+00 5.0000e-02 1.0000e-01 ... 4.0945e+02 4.0950e+02 4.0955e+02] 
	==> ctxt_y  <Pyfhel Ciphertext at 0x215a79a3ae0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>


computation

In [24]:
# Ciphertext-ciphertext ops:
ccSum = ctxt_x + ctxt_y       # Calls HE.add(ctxt_x, ctxt_y, in_new_ctxt=True)
                            #  `ctxt_x += ctxt_y` for inplace operation
ccSub = ctxt_x - ctxt_y       # Calls HE.sub(ctxt_x, ctxt_y, in_new_ctxt=True)
                            #  `ctxt_x -= ctxt_y` for inplace operation
ccMul = ctxt_x * ctxt_y       # Calls HE.multiply(ctxt_x, ctxt_y, in_new_ctxt=True)
                            #  `ctxt_x *= ctxt_y` for inplace operation
cSq   = ctxt_x**2            # Calls HE.square(ctxt_x, in_new_ctxt=True)
                            #  `ctxt_x **= 2` for inplace operation
cNeg  = -ctxt_x              # Calls HE.negate(ctxt_x, in_new_ctxt=True)
                            #
# cPow  = ctxt_x**3          # pow Not supported in CKKS
cRotR = ctxt_x >> 2          # Calls HE.rotate(ctxt_x, k=2, in_new_ctxt=True)
                            #  `ctxt_x >>= 2` for inplace operation
cRotL = ctxt_x << 2          # Calls HE.rotate(ctxt_x, k=-2, in_new_ctxt=True)
                            #  `ctxt_x <<= 2` for inplace operation

# Ciphetext-plaintext ops
cpSum = ctxt_x + ptxt_y       # Calls HE.add_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
                            # `ctxt_x += ctxt_y` for inplace operation
cpSub = ctxt_x - ptxt_y       # Calls HE.sub_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
                            # `ctxt_x -= ctxt_y` for inplace operation
cpMul = ctxt_x * ptxt_y       # Calls HE.multiply_plain(ctxt_x, ptxt_y, in_new_ctxt=True)
                            # `ctxt_x *= ctxt_y` for inplace operation


print("Secure operations")
print(" Ciphertext-ciphertext: ")
print("->\tctxt_x + ctxt_y = ccSum: ", ccSum)
print("->\tctxt_x - ctxt_y = ccSub: ", ccSub)
print("->\tctxt_x * ctxt_y = ccMul: ", ccMul)
print(" Single ciphertext: ")
print("->\tctxt_x**2      = cSq  : ", cSq  )
print("->\t- ctxt_x       = cNeg : ", cNeg )
print("->\tctxt_x >> 4    = cRotR: ", cRotR)
print("->\tctxt_x << 4    = cRotL: ", cRotL)
print(" Ciphertext-plaintext: ")
print("->\tctxt_x + ptxt_y = cpSum: ", cpSum)
print("->\tctxt_x - ptxt_y = cpSub: ", cpSub)
print("->\tctxt_x * ptxt_y = cpMul: ", cpMul)

Secure operations
 Ciphertext-ciphertext: 
->	ctxt_x + ctxt_y = ccSum:  <Pyfhel Ciphertext at 0x215ff476310, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x - ctxt_y = ccSub:  <Pyfhel Ciphertext at 0x215ff45a810, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x * ctxt_y = ccMul:  <Pyfhel Ciphertext at 0x215ff476810, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>
 Single ciphertext: 
->	ctxt_x**2      = cSq  :  <Pyfhel Ciphertext at 0x215ff476950, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>
->	- ctxt_x       = cNeg :  <Pyfhel Ciphertext at 0x215a79a39a0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x >> 4    = cRotR:  <Pyfhel Ciphertext at 0x215fefc3e00, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x << 4    = cRotL:  <Pyfhel Ciphertext at 0x215a79a47c0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
 Ciphertext-plaintext: 
->	ctxt_x + ptxt_y = cpSum:  <Pyfhel Ciphertext at 0x215a79a4b80, scheme=ckks, size=2/2, scale_bi

relinearize

In [20]:
print("Relinearization-> Right after each multiplication.")
print(f"ccMul before relinearization (size {ccMul.size()}): {ccMul}")
HE.relinKeyGen()
~ccMul    # Equivalent to HE.relinearize(ccMul). Relin always happens in-place.
print(f"ccMul after relinearization (size {ccMul.size()}): {ccMul}")

Relinearization-> Right after each multiplication.
ccMul before relinearization (size 3): <Pyfhel Ciphertext at 0x215ff476950, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>
ccMul after relinearization (size 2): <Pyfhel Ciphertext at 0x215ff476950, scheme=ckks, size=2/3, scale_bits=60, mod_level=1>


In [21]:
_r = lambda x: np.round(x, decimals=6)[:4]
print(f"Securely multiplying {n_mults} times!")
for step in range(1,n_mults+1):
    ctxt_x *= ctxt_y    # Multiply in-place --> implicit align_mod_n_scale()
    ctxt_x = ~(ctxt_x)  # Always relinearize after each multiplication!
    print(f"\tStep {step}:  res {_r(HE.decryptFrac(ctxt_x))}")
try:
    ctxt_x *= ctxt_y
except ValueError as e:
    assert str(e)=='scale out of bounds'
    print(f"If we multiply further we get: {str(e)}")

Securely multiplying 8 times!
	Step 1:  res [-0.     0.005  0.02   0.045]
	Step 2:  res [0.       0.00025  0.002    0.006751]
	Step 3:  res [0.000e+00 1.200e-05 2.000e-04 1.009e-03]
	Step 4:  res [ 0.001099  0.       -0.000427  0.000732]
	Step 5:  res [-0.234375  0.0625   -0.40625   0.21875 ]
	Step 6:  res [-148.    2.  -94.  -86.]
	Step 7:  res [-6.93320454e+19 -2.06794841e+19 -1.26459584e+19 -3.08052415e+19]
	Step 8:  res [ 5.84743869e+10 -5.73770284e+09  7.18475662e+08 -2.43026831e+10]
If we multiply further we get: scale out of bounds


 Rescaling & Mod Switching

In [25]:
#  1. Mean
c_mean = (ctxt_x + ctxt_y) / 2
#  2. MSE
c_mse_1 = ~((ctxt_x - c_mean)**2)
c_mse_2 = (~(ctxt_y - c_mean)**2)
c_mse = (c_mse_1 + c_mse_2)/ 3
#  3. Cumulative sum
c_mse += (c_mse << 1)
c_mse += (c_mse << 2)  # element 0 contains the result
print("\n5. Rescaling & Mod Switching.")
print("->\tMean: ", c_mean)
print("->\tMSE_1: ", c_mse_1)
print("->\tMSE_2: ", c_mse_2)
print("->\tMSE: ", c_mse)


5. Rescaling & Mod Switching.
->	Mean:  <Pyfhel Ciphertext at 0x215daaa6590, scheme=ckks, size=2/2, scale_bits=60, mod_level=1>
->	MSE_1:  <Pyfhel Ciphertext at 0x215daaa6630, scheme=ckks, size=2/3, scale_bits=60, mod_level=2>
->	MSE_2:  <Pyfhel Ciphertext at 0x215daa961d0, scheme=ckks, size=2/3, scale_bits=60, mod_level=2>
->	MSE:  <Pyfhel Ciphertext at 0x215a79a44a0, scheme=ckks, size=2/2, scale_bits=60, mod_level=3>


Decrypt & Decode

In [26]:
r_x    = HE.decryptFrac(ctxt_x)
r_y    = HE.decryptFrac(ctxt_y)
rccSum = HE.decryptFrac(ccSum)
rccSub = HE.decryptFrac(ccSub)
rccMul = HE.decryptFrac(ccMul)
rcSq   = HE.decryptFrac(cSq  )
rcNeg  = HE.decryptFrac(cNeg )
rcRotR = HE.decryptFrac(cRotR)
rcRotL = HE.decryptFrac(cRotL)
rcpSum = HE.decryptFrac(cpSum)
rcpSub = HE.decryptFrac(cpSub)
rcpMul = HE.decryptFrac(cpMul)
rmean  = HE.decryptFrac(c_mean)
rmse   = HE.decryptFrac(c_mse)

# Note: results are approximate! if you increase the decimals, you will notice
#  the errors
_r = lambda x: np.round(x, decimals=3)
print("6. Decrypting results")
print(" Original ciphertexts: ")
print("   ->\tctxt_x --(decr)--> ", _r(r_x))
print("   ->\tctxt_y --(decr)--> ", _r(r_y))
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt_x + ctxt_y = ccSum --(decr)--> ", _r(rccSum))
print("   ->\tctxt_x - ctxt_y = ccSub --(decr)--> ", _r(rccSub))
print("   ->\tctxt_x * ctxt_y = ccMul --(decr)--> ", _r(rccMul))
print(" Single ciphertext: ")
print("   ->\tctxt_x**2      = cSq   --(decr)--> ", _r(rcSq  ))
print("   ->\t- ctxt_x       = cNeg  --(decr)--> ", _r(rcNeg ))
print("   ->\tctxt_x >> 4    = cRotR --(decr)--> ", _r(rcRotR))
print("   ->\tctxt_x << 4    = cRotL --(decr)--> ", _r(rcRotL))
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt_x + ptxt_y = cpSum --(decr)--> ", _r(rcpSum))
print("   ->\tctxt_x - ptxt_y = cpSub --(decr)--> ", _r(rcpSub))
print("   ->\tctxt_x * ptxt_y = cpMul --(decr)--> ", _r(rcpMul))
print(" Mean Squared error: ")
print("   ->\tmean(ctxt_x, ctxt_y) = c_mean --(decr)--> ", _r(rmean))
print("   ->\tmse(ctxt_x, ctxt_y)  = c_mse  --(decr)--> ", _r(rmse))

6. Decrypting results
 Original ciphertexts: 
   ->	ctxt_x --(decr)-->  [0.000e+00 1.000e-01 2.000e-01 ... 8.189e+02 8.190e+02 8.191e+02]
   ->	ctxt_y --(decr)-->  [-0.0000e+00  5.0000e-02  1.0000e-01 ...  4.0945e+02  4.0950e+02
  4.0955e+02]
 Ciphertext-ciphertext Ops: 
   ->	ctxt_x + ctxt_y = ccSum --(decr)-->  [0.00000e+00 1.50000e-01 3.00000e-01 ... 1.22835e+03 1.22850e+03
 1.22865e+03]
   ->	ctxt_x - ctxt_y = ccSub --(decr)-->  [0.0000e+00 5.0000e-02 1.0000e-01 ... 4.0945e+02 4.0950e+02 4.0955e+02]
   ->	ctxt_x * ctxt_y = ccMul --(decr)-->  [0.00000000e+00 5.00000000e-03 2.00000000e-02 ... 3.35298605e+05
 3.35380500e+05 3.35462403e+05]
 Single ciphertext: 
   ->	ctxt_x**2      = cSq   --(decr)-->  [0.00000000e+00 1.00000000e-02 4.00000000e-02 ... 6.70597210e+05
 6.70760993e+05 6.70924809e+05]
   ->	- ctxt_x       = cNeg  --(decr)-->  [-0.000e+00 -1.000e-01 -2.000e-01 ... -8.189e+02 -8.190e+02 -8.191e+02]
   ->	ctxt_x >> 4    = cRotR --(decr)-->  [819.  819.1  -0.  ... 818.7 818.8 

Profiling

In [29]:
%timeit t = arr_x + arr_y
%timeit t = arr_x * arr_y
%timeit t = arr_x**2
%timeit t = -arr_x

3.64 µs ± 59.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.72 µs ± 44.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.24 µs ± 37.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.67 µs ± 70.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
# Ciphertext-ciphertext ops:
%timeit ccSum = ctxt_x + ctxt_y
%timeit ccSub = ctxt_x - ctxt_y

%timeit ccMul = ctxt_x * ctxt_y
%timeit cSq   = ctxt_x**2

%timeit cNeg  = -ctxt_x

# cPow  = ctxt_x**3          # pow Not supported in CKKS
%timeit cRotR = ctxt_x >> 2
%timeit cRotL = ctxt_x << 2

623 µs ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
626 µs ± 5.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.54 ms ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.09 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
460 µs ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
29.8 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
29.9 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
# Ciphetext-plaintext ops
%timeit cpSum = ctxt_x + ptxt_y
%timeit cpSub = ctxt_x - ptxt_y
%timeit cpMul = ctxt_x * ptxt_y

368 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
386 µs ± 21.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.14 ms ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%timeit ccMul = ctxt_x * ctxt_y; ~ccMul;

32 ms ± 784 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
